In [13]:
from preprocessing import transform_data_for_rnn
from data_scraper import scrape_udp_data
import tensorflow as tf
import numpy as np

In [14]:
# Scrape data from UDP files
# Output saved in "udp_data.pkl"
scrape_udp_data(r"C:\\Users\\chros\\Documents\\Deep Racing\\australia_run1\\udp_data\\motion_packets", r"C:\\Users\\chros\\Documents\\Deep Racing\\australia_run1\\udp_data\\car_telemetry_packets", filename="udp_data.pkl")

Pickle file already exists


In [15]:
# Preprocess scraped UDP data
# Output saved in "X_data.pkl" and "y_data.pkl"
buffer_size = 5
X_data, y_data = transform_data_for_rnn("udp_data.pkl", buffer_size=buffer_size)

In [16]:
# Validate data shapes
print(X_data.shape)
print(y_data.shape)

(174280, 75)
(174280, 15)


In [17]:
# Create dataset
shuffle_buffer_size = 1000
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((X_data, y_data)).shuffle(shuffle_buffer_size).batch(batch_size)
test_dataset = dataset.take(2000)
train_dataset = dataset.skip(2000)

In [18]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]),
  tf.keras.layers.LSTM(32, return_sequences=True),
  tf.keras.layers.LSTM(32),
  tf.keras.layers.Dense(3 * buffer_size)
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, None, 1)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 32)          4352      
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                495       
Total params: 13,167
Trainable params: 13,167
Non-trainable params: 0
_________________________________________________________________


In [19]:
checkpoint_path = "state_estimator_1.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True)
model.compile(loss=tf.keras.losses.Huber(), optimizer='adam', metrics=["mse"])
history = model.fit(train_dataset, validation_data=test_dataset, epochs=100, callbacks=[checkpoint])

Epoch 1/100
3046/3447 [=========================>....] - ETA: 15s - loss: 283.1289 - mse: 159561.5883

KeyboardInterrupt: 

In [7]:
model.load_weights("state_estimator.h5")

In [12]:
metric = tf.keras.metrics.MeanSquaredError()

train_predictions = model.predict(train_dataset)
train_labels = np.concatenate([y for x,y in train_dataset], axis=0) # extracts labels from test dataset
metric.update_state(train_predictions, train_labels)
print("Training Set MSE:", metric.result().numpy())

val_predictions = model.predict(test_dataset)
val_labels = np.concatenate([y for x,y in test_dataset], axis=0) # extracts labels from test dataset
metric.update_state(val_predictions, val_labels)
print("Validation Set MSE:", metric.result().numpy())

Training Set MSE: 54552.42
Validation Set MSE: 40448.473
